In [24]:
#### this notebook allows u to cache the planning prompt.

import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
from google import genai
from prompts.granular_tooling_capabilities import GRANULAR_TOOLING_CAPABILITIES as prompt
from ai_service_models import GranularPlanningResponse
cache_name = None
from pydantic import BaseModel
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any, List
from enum import Enum

from google.genai import types

client_gemini = genai.Client(api_key=GEMINI_API_KEY)
function_granular_planning_response = types.FunctionDeclaration(
    name='Create_Granular_Planning_Response',
    description='Create a granular planning response for the user query, specifying exact tool function IDs needed.',
    parameters_json_schema=GranularPlanningResponse.model_json_schema()
)

tool = types.Tool(function_declarations=[function_granular_planning_response])

cache_type_prompt = client_gemini.caches.create(
    model = "gemini-2.5-flash",
    config=types.CreateCachedContentConfig(
      display_name='type_prompt', # used to identify the cache
      system_instruction=(
        """You are an expert task planner with deep knowledge of available tools.

            **Your goal:** Analyze the user's request and determine:
            1. **Query Type**: Is this a 'command' (task to execute) or 'conversational' (no action needed)?
            2. **Tooling Need**: Does this require any tools, or can it be answered conversationally?
            3. **Required Tools**: If tools are needed, specify EXACTLY which tool function IDs are required. Be precise and minimal.

            **CRITICAL**: Only include tools that are ACTUALLY needed for THIS specific task. Don't include tools "just in case." However, consider tools that need to be used in tandem to accomplish the task.

            **IMPORTANT**: If multiple tools are needed, list them all and explain how they work together to complete the task.
            **IMPORTANT**: we do not consider capabilities such as "Transcribing the contents" of an image, 'Translating the contents' of an email, 'transcribing an audio file', or 'summarizing a document' as separate tools. These are capabilities that are part of the core AI functionality, and do not require a separate tool. The tools listed here are for external integrations, or for specific actions that require a distinct function call. Such capabilities can be handled by the AI directly, without needing to invoke a separate tool. These capabilities are always available, and you can always do them. 

            the tooling capabilities are detailed in the system prompt.
            Consider the conversation context. If a task was just completed, the user might be responding conversationally.
            """
      ),
      tools=[tool],
      tool_config=types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(mode='ANY')
        ),

      # response_mime_type='application/json',
      # response_schema=TargetTypeAnnotation,
      contents=[prompt],
      ttl="600s",  # 15 days
  )
)
cache_name = cache_type_prompt.name

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [25]:
cache_name

'cachedContents/bzft7d0tw1kfmr0or2ttyj4e730zwr1278uf2m9w'

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI


planning_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GEMINI_API_KEY, thinking_budget=0, cached_content=cache_name)
# planning_llm = planning_llm.with_structured_output(GranularPlanningResponse)
response = planning_llm.invoke("Hi there")

In [13]:
caches = client_gemini.caches.list()
for c in caches:
    client_gemini.caches.delete(name=c.name)

In [20]:
cache_name

'cachedContents/yt3g3bw9y4ezmu6a5n7e1wjkhdrkzcgwpt68pq95'